### GAP Data Analytics, Data Retrieval

This Jupyter Notebook allows for automation in the process of extracting GitHub statistics relevant for the redistribution of the GAP programming language. To extract data from the PyGithub API, it is first necessary to install the PyGitHub library. This library provides a Python wrapper for the GitHub REST API.

In [ ]:
# Import required libraries and packages
from github import Github

# Import modules from other project scripts
from utils import get_github_token

### Connecting to GitHub token

Connecting to GitHub and verifying the user GitHub token is done through storing the access token as an environment variable. This way, the access token is not exposed in the script. The function for getting the token is imported from the utils file in the project.

In [ ]:
# Get the GitHub access token and create instance of the GitHub class
github_token = get_github_token()
if github_token:
    g = Github(github_token)